In [ ]:
!pip install numpy==1.26.4 scipy==1.13.1 gpytoolbox==0.2.0 polyscope==2.2.1 libigl

## Introduction to Mesh Parameterization: Exercises
In this notebook you will perform the same analysis you did in 101 on more complex meshes, and try your hand at more complicated parameterization techniques. 

### Part 1: Fixed Boundary Parameterization and Distortion Analysis ###
In this folder you are given 2 meshes -- halfbunny.obj and ogre.obj. Load each of these meshes using gpytoolbox and use the code from notebook 101 to perform the analyses in the next few code blocks. 

In [ ]:
# Loading required packages
import numpy as np
import polyscope as ps

#### 1.1 halfbunny.obj

In [ ]:
### TODO: use gpytoolbox to load in halfbunny.obj


### TODO: Use the below code copied from notebook 101 to get the boundary and non-boundary edges of the imported mesh
# from igl import boundary_loop

# bnd = boundary_loop(MESH_FACES)
# boundary_idxs = list(sorted(bnd))

# NOTE: pin the boundary to a circle -- can no longer define this by hand
# from igl import map_vertices_to_circle
# boundary_positions = map_vertices_to_circle(MESH_VERTICES, bnd).astype(MESH_VERTICES.dtype)
# NOTE: Resort the positions to match the order of boundary_idxs
# boundary_positions = boundary_positions[np.argsort(bnd)]

# pred_idxs = np.array([i for i in range(MESH_VERTICES.shape[0]) if i not in boundary_idxs])

# # Get edge array
# from collections import defaultdict
# edges = defaultdict(int)
# for f in MESH_FACES:
#     for i in range(3):
#         if f[i] > f[(i+1)%3]:
#             edges[(f[(i+1)%3], f[i])] += 1
#         else:
#             edges[(f[i], f[(i+1)%3])] += 1

# # Valid edges are the ones that are shared by two faces
# tot_edges = np.array(list(edges.keys()))
# valid_edges = np.array([k for k, v in edges.items() if v == 2])
# boundary_edges = np.array([k for k, v in edges.items() if v == 1])

Note in the example code that the process for computing the fixed boundary parameterizations for these meshes will be almost exactly the same as in exercise 101, except it is no longer so trivial to define the boundary positions. Instead, we will make use of libigl to fix the boundary vertices to a circle (standard convex domain). This is done using these additional lines in the example code

```
from igl import map_vertices_to_circle
boundary_positions = map_vertices_to_circle(MESH_VERTICES, bnd).astype(MESH_VERTICES.dtype)
boundary_positions = boundary_positions[np.argsort(bnd)]
```

In [ ]:
# TODO: Copy over the setup_parameterization_matrices() function from notebook 101 and use it to compute
# 1) The Tutte parameterization (all weights of 1)
# 2) The Mean Value weights parameterization (see formula and code for computing the mean value weights from notebook 101)

In [ ]:
# TODO: Visualize the mesh and the computed UV maps using polyscope

In [ ]:
# TODO: Copy over the get_jacobian() function from notebook 101 and use it to
# compute the area, conformal, and isometric distortion for each parameterization.

In [ ]:
# TODO: Visualize the computed distortion energies using polyscope

#### 1.2 ogre.obj
Note that this is a significantly larger mesh so expect `np.linalg.solve()` to take a few minutes to run! 

In [ ]:
# TODO: use gpytoolbox to load in ogre.obj


# TODO: Use the below code copied from notebook 101 to get the boundary and non-boundary edges of the imported mesh
# from igl import boundary_loop

# bnd = boundary_loop(MESH_FACES)
# boundary_idxs = list(sorted(bnd))

# NOTE: pin the boundary to a circle -- can no longer define this by hand
# from igl import map_vertices_to_circle
# boundary_positions = map_vertices_to_circle(MESH_VERTICES, bnd).astype(MESH_VERTICES.dtype)
# NOTE: Resort the positions to match the order of boundary_idxs
# boundary_positions = boundary_positions[np.argsort(bnd)]
# pred_idxs = np.array([i for i in range(pyramid_vs.shape[0]) if i not in boundary_idxs])

# # Get edge array
# from collections import defaultdict
# edges = defaultdict(int)
# for f in MESH_FACES:
#     for i in range(3):
#         if f[i] > f[(i+1)%3]:
#             edges[(f[(i+1)%3], f[i])] += 1
#         else:
#             edges[(f[i], f[(i+1)%3])] += 1

# # Valid edges are the ones that are shared by two faces
# tot_edges = np.array(list(edges.keys()))
# valid_edges = np.array([k for k, v in edges.items() if v == 2])
# boundary_edges = np.array([k for k, v in edges.items() if v == 1])

In [ ]:
# TODO: Copy over the setup_parameterization_matrices() function from notebook 101 and use it to compute
# 1) The Tutte parameterization (all weights of 1)
# 2) The Mean Value weights parameterization (see formula and code for computing the mean value weights from notebook 101)

In [ ]:
# TODO: Visualize the mesh and the computed UV maps using polyscope

In [ ]:
# TODO: Copy over the get_jacobian() function from notebook 101 and use it to
# compute the area, conformal, and isometric distortion for each parameterization.

In [ ]:
# TODO: Visualize the computed distortion energies using polyscope

### Part 2: LSCM and ARAP ###
In this part you will use two more advanced parameterization techniques to flatten the same meshes. 

These two methods are Least Squares Conformal Maps [(LSCM)](https://www.cs.jhu.edu/~misha/Fall09/Levy02.pdf) and As-Rigid-As-Possible Mesh Parameterization [(ARAP)](https://cs.harvard.edu/~sjg/papers/arap.pdf). 

As you will see, these are two examples of **free boundary** methods (though LSCM technically requires two vertices to be pinned), which allows the boundary to move independently to perform the desired distortion minimization. 

#### 2.1 Least Squares Conformal Maps [(LSCM)](https://www.cs.jhu.edu/~misha/Fall09/Levy02.pdf)
As the name implies, LSCM is a **conformal** method, meaning it aims to minimize the conformal (angular) distortion of the parameterization, using a least squares solve. Deriving and computing this method by hand is beyond the scope of this exercise, so we will be using libigl's `lscm()` function to do the computation for us. 

One important note is that LSCM requires two vertices to be pinned in the plane to make the least-squared system well-determined (so there is a unique solution). Technically any two vertices can be chosen, but in practice vertices at the opposite end of a boundary loop are usually the best choice for method performance. The code commented below gives an example of computing the LSCM parameterization using libigl. 

In [ ]:
# TODO: Use the below example code to compute the LSCM parameterization of halfbunny.obj and ogre.obj
# from igl import boundary_loop, lscm

# bdry = boundary_loop(mesh.faces)

# b = np.array([bdry[0], bdry[int(len(bdry)/2)]], dtype="int")
# bc = np.array([[0, 0], [1, 1]], dtype=np.float32)
# succ, lscm_uv, error = lscm(mesh.vertices, mesh.faces, b, bc)

In [ ]:
# TODO: Compute the area, conformal, and isometric distortion for the LSCM parameterization of halfbunny.obj and ogre.obj

In [ ]:
# TODO: Compare the distortions of the LSCM results against the Tutte and Mean Value weights parameterizations
# The LSCM conformal result should be close to 0. What about the area and isometric distortions?

In [ ]:
# TODO: Visualize the distortion values using Polyscope

#### 2.2 As-Rigid-As-Possible Mesh Parameterization [(ARAP)](https://cs.harvard.edu/~sjg/papers/arap.pdf)
The ARAP method aims to minimize isometric distortion (both area and angles), using a non-linear algorithm which alternates between local and global optimization steps. The method requires an initial UV map as input, so we use a harmonic parameterization as an initial guess. 

In [ ]:
# TODO: Use the below example code to compute the ARAP parameterization of halfbunny.obj and ogre.obj
# from igl import ARAP, boundary_loop, harmonic, map_vertices_to_circle
# bnd = boundary_loop(mesh.faces)
# bnd_uv = map_vertices_to_circle(mesh.vertices, bnd).astype(mesh.vertices.dtype)
# initial_uv = harmonic(mesh.vertices, mesh.faces, bnd, bnd_uv, 1)
# arap = ARAP(mesh.vertices, mesh.faces, 2, np.zeros(0), with_dynamics=True)
# arap_uv = arap.solve(np.zeros((0,0)), initial_uv)

In [ ]:
# TODO: Compute the area, conformal, and isometric distortion for the LSCM parameterization of halfbunny.obj and ogre.obj

In [ ]:
# TODO: Compare the distortions of the ARAP results against the LSCM parameterization

In [ ]:
# TODO: Visualize the distortion values using Polyscope